### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

#Create columns to indicate whether students have passing grades and add to student data frame
student_data["passing_reading"] = student_data["reading_score"] >= 70
student_data["passing_math"] = student_data["math_score"] >= 70
student_data["overall_passing"] = (student_data["reading_score"] >= 70) & (student_data["math_score"] >= 70)

student_data

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,passing_reading,passing_math,overall_passing
0,0,Paul Bradley,M,9th,Huang High School,66,79,False,True,False
1,1,Victor Smith,M,12th,Huang High School,94,61,True,False,False
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,True,False,False
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,False,False,False
4,4,Bonnie Ray,F,9th,Huang High School,97,84,True,True,True
...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,True,True,True
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,True,True,True
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,True,True,True
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,True,True,True


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#Count Schools
school_count = school_data["school_name"].count()

#Sum students
student_count = school_data["size"].sum()

#Sum budget
district_budget = school_data["budget"].sum()

#Average all math scores
district_math_avg = student_data["math_score"].mean()

#Average all reading scores
district_reading_avg = student_data["reading_score"].mean()

#Calculate percent passing math
passing_math_perc = (np.sum(student_data["passing_math"] == True) / student_count) * 100

#Calculate percent passing reading

passing_reading_perc = (np.sum(student_data["passing_reading"] == True) / student_count) * 100

#Calculate passing overall

passing_overall_perc = (np.sum(student_data["overall_passing"] == True) / student_count) * 100

#Pass all summary values above into a data frame
district_summary_df = pd.DataFrame({
    "School Count": [school_count],
    "Student Count": student_count,
    "District Budget": district_budget,
    "Distrct Math Average": district_math_avg,
    "District Reading Average": district_reading_avg,
    "% Students Passing Math": passing_math_perc,
    "% Students Passing Reading": passing_reading_perc,
    "% Students Passing": passing_overall_perc
})

district_summary_df

,School Count,Student Count,District Budget,Distrct Math Average,District Reading Average,% Students Passing Math,% Students Passing Reading,% Students Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [7]:
#Pull data that does not need to be changed into a new data frame to start
school_summary_df = school_data[["school_name", "type", "size", "budget"]]

#Aphabetize schools (important when adding in groupby statements)
school_summary_df.sort_values("school_name").reset_index(drop=True)

#Caluclate budget per student and add to data frame
school_summary_df["budget_per_student"] = school_summary_df["budget"] / school_summary_df["size"]


#Create a groupby object by school to pull out scores
groupby_school = student_data.groupby(["school_name"])


#Use groupby school object to average test scores
avg_math_by_school = groupby_school["math_score"].mean()
avg_reading_by_school = groupby_school["reading_score"].mean()

#Place scores in a data frame
test_avgs_df = pd.DataFrame({"avg_math": avg_math_by_school, "avg_reading": avg_reading_by_school})

#Join partial summary table with socres table on school name
summary_with_scores = pd.merge(school_summary_df, test_avgs_df, on="school_name")

#Transition to percent work
#filter entire student data table for only those that passed math
passed_math = student_data.loc[student_data["passing_math"] == True, :]

#Group by school and count the number of students who passed
math_pass_by_school = passed_math.groupby(["school_name"]).count()

#repeat the above two steps for reading scores
passed_reading = student_data.loc[student_data["passing_reading"] == True, :]

reading_pass_by_school = passed_reading.groupby(["school_name"]).count()

#do the same for overall scores
passed_overall = student_data.loc[student_data["overall_passing"] == True, :]

overall_pass_by_school = passed_overall.groupby(["school_name"]).count()

#Combine the above three count series into a data frame
passing_df = pd.DataFrame({"passing_math_count": math_pass_by_school["passing_math"], 
                          "passing_reading_count": reading_pass_by_school["passing_reading"], 
                          "overall_passing_count": overall_pass_by_school["overall_passing"]})


#index with school so that I can divide the columns
indexed_school_summary_df = school_summary_df.set_index(school_summary_df["school_name"])

#create columns to show percent passing by dividing each count by school size and inserting into the data frame
passing_df["passing_math_perc"] = (passing_df["passing_math_count"] / indexed_school_summary_df["size"]) * 100

passing_df["passing_reading_perc"] = (passing_df["passing_reading_count"] / indexed_school_summary_df["size"]) * 100

passing_df["passing_overall_perc"] = (passing_df["overall_passing_count"] / indexed_school_summary_df["size"]) * 100

#Join half-completed summary from above to the percent passing df
full_summary_df = pd.merge(summary_with_scores, passing_df, on="school_name")

#Delete columns for passing counts
del full_summary_df["passing_math_count"]
del full_summary_df["passing_reading_count"]
del full_summary_df["overall_passing_count"]

sorted_full_summary = full_summary_df.sort_values("school_name").reset_index(drop=True)

sorted_full_summary



,school_name,type,size,budget,budget_per_student,avg_math,avg_reading,passing_math_perc,passing_reading_perc,passing_overall_perc
0,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
3,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
6,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [12]:
#Sort in decending order by overall passing
top_summary = sorted_full_summary.sort_values("passing_overall_perc", ascending=False).reset_index(drop=True)

#Take first 5 values only
top_five_summary = top_summary.head()

top_five_summary

,school_name,type,size,budget,budget_per_student,avg_math,avg_reading,passing_math_perc,passing_reading_perc,passing_overall_perc
0,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
1,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
2,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
3,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
4,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [13]:
#Sort in ascending order by overall passing
bottom_summary = sorted_full_summary.sort_values("passing_overall_perc").reset_index(drop=True)

#Take first 5 values only
bottom_five_summary = bottom_summary.head()

bottom_five_summary

,school_name,type,size,budget,budget_per_student,avg_math,avg_reading,passing_math_perc,passing_reading_perc,passing_overall_perc
0,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
2,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
4,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [26]:
grouped_by_grades = student_data.groupby(["school_name", "grade"]).mean()

grouped_by_grades

Student ID  reading_score  math_score  \
school_name           grade                                            
Bailey High School    10th   20365.058918      80.907183   76.996772   
                      11th   20345.148681      80.945643   77.515588   
                      12th   20386.724708      80.912451   76.492218   
                      9th    20344.481481      81.303155   77.083676   
Cabrera High School   10th   16909.487124      84.253219   83.154506   
                      11th   16955.047718      83.788382   82.765560   
                      12th   16924.570681      84.287958   83.277487   
                      9th    16969.634470      83.676136   83.094697   
Figueroa High School  10th    4332.703801      81.408912   76.539974   
                      11th    4424.478138      80.640339   76.884344   
                      12th    4414.922705      81.384863   77.151369   
                      9th     4397.878505      81.198598   76.403037   
Ford High School      10th   36122.879944      81.262712   77.672316   
                      11th   36212.534143      80.403642   76.918058   
                      12th   36153.562152      80.662338   76.179963   
                      9th    36170.595438      80.632653   77.361345   
Griffin High School   10th   12983.711823      83.706897   84.229064   
                      11th   12983.038781      84.288089   83.842105   
                      12th   12977.143836      84.013699   83.356164   
                      9th    13031.305623      83.369193   82.044010   
Hernandez High School 10th    9930.783211      80.660147   77.337408   
                      11th    9948.641544      81.396140   77.136029   
                      12th    9978.563966      80.857143   77.186567   
                      9th     9928.620839      80.866860   77.438495   
Holden High School    10th   23076.342105      83.324561   83.429825   
                      11th   23046.883495      83.815534   85.000000   
                      12th   23041.108434      84.698795   82.855422   
                      9th    23068.314961      83.677165   83.787402   
Huang High School     10th    1450.796610      81.512386   75.908735   
                      11th    1458.094313      81.417476   76.446602   
                      12th    1485.035897      80.305983   77.225641   
                      9th     1445.726303      81.290284   77.027251   
Johnson High School   10th   32467.993480      80.773431   76.691117   
                      11th   32378.549249      80.616027   77.491653   
                      12th   32414.657051      81.227564   76.863248   
                      9th    32399.975714      81.260714   77.187857   
Pena High School      10th   23769.208000      83.612000   83.372000   
                      11th   23742.605469      84.335938   84.328125   
                      12th   23733.342541      84.591160   84.121547   
                      9th    23766.127273      83.807273   83.625455   
Rodriguez High School 10th   28050.171154      80.629808   76.612500   
                      11th   28048.515905      80.864811   76.395626   
                      12th   28023.722433      80.376426   77.690748   
                      9th    28017.408076      80.993127   76.859966   
Shelton High School   10th    6748.301339      83.441964   82.917411   
                      11th    6742.453883      84.373786   83.383495   
                      12th    6740.566038      82.781671   83.778976   
                      9th     6750.615094      84.122642   83.420755   
Thomas High School    10th   38346.368171      84.254157   83.087886   
                      11th   38345.636145      83.585542   83.498795   
                      12th   38360.289941      83.831361   83.497041   
                      9th    38356.793926      83.728850   83.590022   
Wilson High School    10th   14884.930693      84.021452   83.724422   
                      11th   14855.539232      83.764608   83.195326   

## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type